# Cliopatria viewer

1. To get started, download a copy of the Cliopatria dataset from here: `[INSERT LINK]`
2. Move the downloaded dataset to an appropriate location on your machine and pass in the paths in the code cell below and run
3. Run the subsequent cells of the notebook
4. Play around with both the GeoDataFrame (gdf) and the rendered map


In [1]:
cliopatria_geojson_path = "../data/cliopatria_composite_unique_nonsimplified.geojson_06052024/cliopatria_composite_unique_nonsimplified.geojson"
cliopatria_json_path = "../data/cliopatria_composite_unique_nonsimplified.geojson_06052024/cliopatria_composite_unique_nonsimplified_name_years.json"

In [3]:
from map_functions import cliopatria_gdf, display_map

In [4]:
# Load the Cliopatria data to a GeoDataFrame including end years for each shape
gdf = cliopatria_gdf(cliopatria_geojson_path, cliopatria_json_path)

# Play with the data on the map

**Notes**
- The slider is a bit buggy, the best way to change year is to enter a year in the box and hit enter. Use minus numbers for BCE.
- The map is also displayed thrice for some reason!
- Initial attempts to implement a play button similar to the website code failed, but that may not be needed here.
- Click the shapes to reveal the polity display names, using the same logic used in the website code - see `map_functions.py`

In [6]:
display_year = 0
display_map(gdf, display_year)

IntText(value=0, description='Year:')

IntSlider(value=0, description='Year:', max=2024, min=-3400)

Output()

/Users/echalstrey/.pyenv/versions/3.11.4/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
